In [70]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import torch

import logging
logger = logging.getLogger(__name__)

In [1]:
# after process data structure for each modality
# acoustic = [[[feature at time 1], [feature at time 2], [...]], [...]]
# linguistic = [[[token_1, token_2, ... at time 1], [token_1, token_2, ... at time 2], [...]], [...]]
# visual = [[[feature at time 1], [feature at time 2], [...]], [...]]

In [3]:
modalities_data_dir = "../../SENDv1-data/features/Train/"

In [63]:
preprocess = {
    'acoustic': lambda df : df.loc[:,' F0semitoneFrom27.5Hz_sma3nz_amean':' equivalentSoundLevel_dBp'],
    'acoustic_timer': lambda df : df.loc[:,' frameTime']
}

class InputFeature:
    
    def __init__(
        self, video_id="",
        acoustic_feature=[],
        linguistic_feature=[],
        visual_feature=[],
        labels=[],
    ):
        self.video_id = video_id
        self.acoustic_feature = acoustic_feature
        self.linguistic_feature = linguistic_feature
        self.visual_feature = visual_feature
        self.labels = labels

In [67]:
def preprocess_SEND_files(
    data_dir,
    time_window_in_sec=5,
    modality_dir_map = {"acoustic": "acoustic-egemaps",  
                        "linguistic": "linguistic-word-level", # we don't load features
                        "visual": "image-raw", # image is nested
                       },
):
    # basically, let us gett all the video ids?
    a_ids = [f.split("_")[0]+"_"+f.split("_")[1] 
             for f in listdir(os.path.join(data_dir, modality_dir_map["acoustic"])) 
             if isfile(os.path.join(data_dir, modality_dir_map["acoustic"], f))]
    l_ids = [f.split("_")[0]+"_"+f.split("_")[1] 
             for f in listdir(os.path.join(data_dir, modality_dir_map["linguistic"])) 
             if isfile(os.path.join(data_dir, modality_dir_map["linguistic"], f))]
    v_ids = [f.split("_")[0]+"_"+f.split("_")[1] 
             for f in listdir(os.path.join(data_dir, modality_dir_map["visual"])) 
             if f != ".DS_Store"]
    assert len(a_ids) == len(l_ids) and len(l_ids) == len(v_ids)
    assert len(set(a_ids).intersection(set(l_ids))) == len(l_ids)
    assert len(set(a_ids).intersection(set(v_ids))) == len(v_ids)
    
    for video_id in a_ids: # pick any one!
        
        # acoustic features process
        a_file = os.path.join(data_dir, modality_dir_map["acoustic"], f"{video_id}_acousticFeatures.csv")
        a_df = pd.read_csv(a_file)
        a_features = np.array(preprocess["acoustic"](a_df))
        
        # linguistic features process
        
        
        break

In [68]:
preprocess_SEND_files(modalities_data_dir)

[[ 1.620367e+01  7.090776e-02  1.512075e+01 ...  5.500000e-02
   3.500000e-02 -4.161761e+01]
 [ 1.568518e+01  1.482646e-01  1.296304e+01 ...  3.000000e-02
   1.732051e-02 -3.602686e+01]
 [ 2.599282e+01  2.938939e-02  2.552257e+01 ...  0.000000e+00
   0.000000e+00 -2.212005e+01]
 ...
 [ 1.708549e+01  6.185527e-02  1.595142e+01 ...  2.500000e-02
   5.000000e-03 -4.190898e+01]
 [ 1.614124e+01  1.208483e-01  1.449218e+01 ...  0.000000e+00
   0.000000e+00 -3.995140e+01]
 [ 1.736700e+01  1.246059e-01  1.600804e+01 ...  4.000000e-02
   2.000000e-02 -4.237817e+01]]
